In [33]:
import json
from sqlalchemy import UnicodeText
from sqlalchemy.types import TypeDecorator
from sqlalchemy.dialects.mysql import JSON, INTEGER
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Table, Column, String, Enum, TIMESTAMP, TEXT, ForeignKey, text, DATE, create_engine, ForeignKeyConstraint, UniqueConstraint

In [34]:
class JSONType(TypeDecorator):
    impl=UnicodeText
    
    def load_dialect_impl(self, dialect):
        if dialect.name == 'mysql':
            return dialect.type_descriptor(JSON())
        else:
            return dialect.type_descriptor(self.impl)

    def process_bind_param(self, value, dialect):
        if dialect.name == 'mysql':
            return value
        if value is not None:
            value = json.dumps(value)
        return value

    def process_result_value(self, value, dialect):
        if dialect.name == 'mysql':
            return value
        if value is not None:
            value = json.loads(value)
        return value

In [35]:
type(JSONType)

sqlalchemy.sql.visitors.VisitableType

In [36]:
Base = declarative_base()

class Product(Base):
    __tablename__ = 'product'

    product_id = Column(INTEGER, primary_key=True, autoincrement=True)
    name = Column(String(50))
    details = Column(JSONType)

In [42]:
dbparams={'dbhost':'0.0.0.0', \
          'dbuser':'root', \
          'dbpass':'xxxx', \
          'dbname':'igfdb', \
          'driver':'mysql', \
          'connector':'pymysql'}

In [54]:
#dbparams={'dbname':'test.db'} #

In [55]:
from igf_data.igfdb.baseadaptor import BaseAdaptor

In [56]:
base=BaseAdaptor(**dbparams)
session_class=base.get_session_class()

In [57]:
Base.metadata.create_all(base.engine)

In [58]:
base.start_session()

In [59]:
data=[{'name':'seqrun_data','details':{'color': 'red', 'type': 'car', 'max-speed': '400 mph'}}]

In [60]:
data

[{'details': {'color': 'red', 'max-speed': '400 mph', 'type': 'car'},
  'name': 'seqrun_data'}]

In [61]:
base.store_records(table=Product, data=data)
base.commit_session()

In [62]:
p_data=base.fetch_records(query=base.session.query(Product))

In [63]:
p_data

,product_id,name,details
0,1,seqrun_data,"{'type': 'car', 'color': 'red', 'max-speed': '..."


In [64]:
base.close_session()